In [93]:
import os
import re
import pandas as pd

# caminho para a pasta onde estão os arquivos CSV
caminho_pasta = 'dados'
# lista com os dataframes já tratados
dfs = []
# Lista com os códigos de cid-10 pertencentes aos ICSAPS
COD_ICSAPS = ["A37","A36","A33","A34","A35","B26","B06","B05","A95","B16","G000","A170","A19","A150","A151","A152","A153","A160","A161","A162",
          "A154","A155","A156","A157","A158","A159","A163","A164","A165","A166","A167","A168","A169","A171","A172","A173","A174","A175","A176",
          "A177","A178","A179","A18","I00","I01","I02","A51","A52","A53","B50","B51","B52","B53","B54","B77","E86","A00","A01","A02","A03","A04",
          "A05","A06","A07","A08","A09","D50","E40","E41","E42","E43","E44","E45","E46","E50","E51","E52","E53","E54","E55","E56","E57","E58",
          "E59","E60","E61","E62","E63","E64","H66","J00","J01","J02","J03","J06","J31","J13","J14","J153","J154","J158","J159","J181","J45",
          "J46","J20","J21","J40","J41","J42","J43","J47","J44","I10","I11","I20","I50","J81","I63","I64","I65","I66","I67","I69","G45","G46",
          "E100","E101","E110","E111","E120","E121","E130","E131","E140","E141","E102","E103","E104","E105","E106","E107","E108","E112","E113",
          "E114","E115","E116","E117","E118","E122","E123","E124","E125","E126","E127","E128","E132","E133","E134","E135","E136","E137","E138",
          "E142","E143","E144","E145","E146","E147","E148","E109","E119","E129","E139","E149","G40","G41","N10","N11","N12","N30","N34","N390",
          "A46","L01","L02","L03","L04","L08","N70","N71","N72","N73","N75","N76","K25","K26","K27","K28","K920","K921","K922","O23","A50","P350"]

for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.csv'):
        # Ler o arquivo CSV com o pandas
        df = pd.read_csv(os.path.join(caminho_pasta, arquivo), delimiter=';', encoding='ISO-8859-1', low_memory=False)
        # Criar coluna ano_arquivo para identificar arquivo fonte
        df['ano_arquivo'] = re.findall(r'\d+', arquivo)[0]
        # Criar unicidade para não repetir o dado em futuras atualizações
        df['pk_base_origem'] = df['ano_arquivo'] + '_' + df['CONTADOR'].astype(str)
        # Realizar transformação das datas de nascimento e óbito
        df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')
        df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y', errors='coerce')
        # Criando a coluna 'ano_obito'
        df['ano_obito'] = df['DTOBITO'].dt.year.astype(float).astype(pd.Int64Dtype()).astype(str).where(df['DTOBITO'].notna())
        # Criando a coluna 'quadrimestre_obito' usando pd.cut()
        df['quadrimestre_obito'] = pd.cut(df['DTOBITO'].dt.month, bins=[1, 5, 9, 13], labels=[1, 2, 3], right=False)

        # Função para transformar a idade conforme as regras especificadas pela fonte
        def transformar_idade(idade):
            if pd.isnull(idade):
                return None
            elif str(idade).startswith('4'):
                idade_str = str(idade).split('.')[0][1:]
                if idade_str:
                    return int(idade_str)
            return 0
        # Aplicando a transformação na coluna de idade
        df['IDADE'] = df['IDADE'].apply(transformar_idade)

        # Transfromar tipo de óbito
        mapeamento = {1: 'Fetal', 2: 'Não Fetal', 3: 'Ignorado'}
        df['TIPOBITO'] = df['TIPOBITO'].replace(mapeamento)
        # Extrair os 6 primeiros dígitos da coluna CODMUNRES
        df['CODMUNRES'] = df['CODMUNRES'].astype(str).str.slice(stop=6)

        # Função lambda para aplicar a condição
        df['icsaps'] = df['CAUSABAS'].apply(lambda x: 'Sim' if x in COD_ICSAPS else 'Não')

        # Renomear colunas
        df = df.rename(columns={
            'DTOBITO': 'dt_obito',
            'DTNASC':'dt_nascimento',
            'IDADE':'idade',
            'TIPOBITO':'tp_obito',
            'CAUSABAS':'cid10',
            'CODMUNRES':'cd_municipio_residencia'
            })
        # Selecionar coluna desejadas
        df = df[[
            "pk_base_origem","ano_arquivo","ano_obito","quadrimestre_obito","dt_obito",
            "dt_nascimento","idade","tp_obito","cid10","icsaps","cd_municipio_residencia"
            ]]
        
        # adiciona o dataframe à lista de dataframes
        dfs.append(df)

# concatena os dataframes em um único dataframe final
df_group = pd.concat(dfs, ignore_index=True)

In [94]:
# exibe o dataframe final
df_group.head()

,pk_base_origem,ano_arquivo,ano_obito,quadrimestre_obito,dt_obito,dt_nascimento,idade,tp_obito,cid10,icsaps,cd_municipio_residencia
0,2010_1,2010,2010,2,2010-08-06,1945-04-09,65.0,Não Fetal,R98,Não,120040
1,2010_2,2010,2010,2,2010-08-06,1912-01-20,98.0,Não Fetal,I219,Não,120040
2,2010_3,2010,2010,3,2010-10-02,2010-03-17,0.0,Não Fetal,R98,Não,120040
3,2010_4,2010,2010,1,2010-04-07,2010-04-07,0.0,Não Fetal,P969,Não,120042
4,2010_5,2010,2010,2,2010-05-13,1971-08-04,38.0,Não Fetal,X999,Não,120020


In [95]:
df_group.dtypes

pk_base_origem                     object
ano_arquivo                        object
ano_obito                          object
quadrimestre_obito               category
dt_obito                   datetime64[ns]
dt_nascimento              datetime64[ns]
idade                             float64
tp_obito                           object
cid10                              object
icsaps                             object
cd_municipio_residencia            object
dtype: object

In [96]:
df_group.columns

Index(['pk_base_origem', 'ano_arquivo', 'ano_obito', 'quadrimestre_obito',
       'dt_obito', 'dt_nascimento', 'idade', 'tp_obito', 'cid10', 'icsaps',
       'cd_municipio_residencia'],
      dtype='object')

In [97]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12479256 entries, 0 to 12479255
Data columns (total 11 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   pk_base_origem           object        
 1   ano_arquivo              object        
 2   ano_obito                object        
 3   quadrimestre_obito       category      
 4   dt_obito                 datetime64[ns]
 5   dt_nascimento            datetime64[ns]
 6   idade                    float64       
 7   tp_obito                 object        
 8   cid10                    object        
 9   icsaps                   object        
 10  cd_municipio_residencia  object        
dtypes: category(1), datetime64[ns](2), float64(1), object(7)
memory usage: 964.0+ MB


In [98]:
df_group.shape

(12479256, 11)

In [99]:
df_group.isna().sum()

pk_base_origem                  0
ano_arquivo                     0
ano_obito                   66735
quadrimestre_obito          66735
dt_obito                    66735
dt_nascimento              117161
idade                           4
tp_obito                        0
cid10                           0
icsaps                          0
cd_municipio_residencia         0
dtype: int64

In [100]:
df_icsaps = df_group[df_group['icsaps'] == 'Sim']

In [101]:
df_icsaps.shape

(1776514, 11)

In [102]:
df_icsaps.isna().sum()

pk_base_origem                 0
ano_arquivo                    0
ano_obito                   9475
quadrimestre_obito          9475
dt_obito                    9475
dt_nascimento              11962
idade                          0
tp_obito                       0
cid10                          0
icsaps                         0
cd_municipio_residencia        0
dtype: int64

In [111]:
df_icsaps_tratado = df_icsaps.dropna(subset=['dt_nascimento'])
df_icsaps_tratado = df_icsaps_tratado.dropna(subset=['dt_obito'])
df_icsaps_tratado = df_icsaps_tratado.dropna(subset=['idade'])

In [112]:
df_icsaps_tratado.shape

(1755154, 11)

In [113]:
df_icsaps_tratado.isna().sum()

pk_base_origem             0
ano_arquivo                0
ano_obito                  0
quadrimestre_obito         0
dt_obito                   0
dt_nascimento              0
idade                      0
tp_obito                   0
cid10                      0
icsaps                     0
cd_municipio_residencia    0
dtype: int64

In [114]:
# Calcular a diferença em anos entre as colunas de data
# df_icsaps_tratado['idade_dif'] = (df_icsaps_tratado['dt_obito'] - df_icsaps_tratado['dt_nascimento']).dt.days // 365

from dateutil.relativedelta import relativedelta
df_icsaps_tratado['idade_dif'] = df_icsaps_tratado.apply(lambda row: relativedelta(row['dt_obito'], row['dt_nascimento']).years, axis=1)
df_icsaps_tratado['idade'] = df_icsaps_tratado['idade'].astype(int)
df_icsaps_tratado['idade'] = df_icsaps_tratado['idade'].astype(int)

In [115]:
df_icsaps_tratado.isna().sum()

pk_base_origem             0
ano_arquivo                0
ano_obito                  0
quadrimestre_obito         0
dt_obito                   0
dt_nascimento              0
idade                      0
tp_obito                   0
cid10                      0
icsaps                     0
cd_municipio_residencia    0
idade_dif                  0
dtype: int64

In [116]:
df_filtrado = df_icsaps_tratado[df_icsaps_tratado['idade'] != df_icsaps_tratado['idade_dif']]

In [117]:
df_filtrado.head()

,pk_base_origem,ano_arquivo,ano_obito,quadrimestre_obito,dt_obito,dt_nascimento,idade,tp_obito,cid10,icsaps,cd_municipio_residencia,idade_dif
429,2010_430,2010,2010,2,2010-07-04,1908-08-26,0,Não Fetal,I64,Sim,120030,101
794,2010_795,2010,2010,1,2010-02-21,1939-05-14,71,Não Fetal,E126,Sim,120040,70
843,2010_844,2010,2010,1,2010-01-11,1925-04-20,85,Não Fetal,I64,Sim,120038,84
1034,2010_1035,2010,2010,1,2010-01-11,1935-03-31,75,Não Fetal,E120,Sim,120040,74
1097,2010_1098,2010,2010,1,2010-02-11,1942-04-21,68,Não Fetal,N390,Sim,120040,67


In [118]:
df_filtrado.shape

(66434, 12)